Lucía García-Duarte Sáenz
# MFSTC-CNN-LSTM Model

- Architecture: 
  1. User defined number of Conv2D followed by Maxpooling layers
  2. User defined number of LSTM layers
  3. Dense layers

In [ ]:
import os
import sys
import urllib.request
import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input, Flatten, TimeDistributed, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator


## Load the data

- P NxT matrices, which describes the ($T_1, \cdots, T_t$) records over t timesteps for the N stations for each of the P variables.

Data is recorded every hour over 37 stations for 26 years

In [ ]:
mypath = 'C:/Users/lgarc/OneDrive/Documentos/AA UNI/Master/TFM/2. Missing values imputation/temporal imputed data/MOVING AVERAGE'

data_temp = pd.read_csv(mypath + '/data_imputed_air_temp.csv')
data_temp = data_temp.drop(columns=['Unnamed: 0','date'])
data_temp = data_temp.T

data_dew = pd.read_csv(mypath + '/data_imputed_dew_point.csv')
data_dew = data_dew.drop(columns=['Unnamed: 0','date'])
data_dew = data_dew.T

data_wd = pd.read_csv(mypath + '/data_imputed_wd.csv')
data_wd = data_wd.drop(columns=['Unnamed: 0','date'])
data_wd = data_wd.T

data_ws = pd.read_csv(mypath + '/data_imputed_ws.csv')
data_ws = data_ws.drop(columns=['Unnamed: 0','date'])
data_ws = data_ws.T

data_vis = pd.read_csv(mypath + '/data_imputed_visibility.csv')
data_vis = data_vis.drop(columns=['Unnamed: 0','date'])
data_vis = data_vis.T

data_RH = pd.read_csv(mypath + '/data_imputed_RH.csv')
data_RH = data_RH.drop(columns=['Unnamed: 0','date'])
data_RH = data_RH.T

In [ ]:
# Join data

data_all = np.array([data_temp.values, data_dew.values, data_wd.values, data_ws.values, data_vis.values,data_RH.values])
data_all = data_all.reshape((1,6, 37, -1))
data_all = data_all.T

time_len, num_nodes, num_vars, aux = data_all.shape
print("No. of stations:", num_nodes, "No. of variables:", num_vars, "\nNo. of timesteps:", time_len)

data_all.shape

## Prepare the data

In [ ]:
### TRAIN/VAL/TEST SPLIT

test_rate = 0.3
valid_rate = 0.1

time_len = data_all.shape[0]
train_portion = 1 - valid_rate - test_rate
train_size = int(time_len * train_portion)
valid_size = int(time_len * valid_rate)

train_data = data_all[:train_size]
valid_data = data_all[train_size:train_size+valid_size]
test_data = data_all[train_size+valid_size:]
    
print(train_data.shape)
print(valid_data.shape)
print(test_data.shape)

In [ ]:
### SCALE DATA

def scale_data(train_data, valid_data, test_data):
    
    max_speed = np.reshape(np.squeeze(train_data).max(0).max(0), (6, 1))
    min_speed = np.reshape(np.squeeze(train_data).min(0).min(0), (6, 1))
    
    train_scaled = (train_data - min_speed) / (max_speed - min_speed)
    valid_scaled = (valid_data - min_speed) / (max_speed - min_speed)
    test_scaled = (test_data - min_speed) / (max_speed - min_speed)
    
    return train_scaled, valid_scaled, test_scaled

train_scaled, valid_scaled, test_scaled = scale_data(train_data, valid_data, test_data)
#del data_all # to save memory for training

In [ ]:
### PREPARE DATA

seq_len = 24    # change by 24, 24*7, 24*7*2, 24*7*3 (h)
pre_len = 1     # change by 1, 2, 3 (h)

class CustomGen(TimeseriesGenerator):
    def __getitem__(self, idx):
        x,y = super().__getitem__(idx)
        y = y[:,:,0,:] # modify the generator to get only temperature data in the output
        return x, y

if pre_len == 1:
    train_generator = CustomGen(train_scaled, train_scaled, length=seq_len, batch_size = 1)
    valid_generator = CustomGen(valid_scaled, valid_scaled, length=seq_len, batch_size = 1)
    test_generator = CustomGen(test_scaled, test_scaled, length=seq_len, batch_size = 1)
else:
    train_generator = CustomGen(train_scaled[:-pre_len+1,:,:,:], train_scaled[pre_len-1:,:,:,:], length=seq_len, batch_size = 1)
    valid_generator = CustomGen(valid_scaled[:-pre_len+1,:,:,:], valid_scaled[pre_len-1:,:,:,:], length=seq_len, batch_size = 1)
    test_generator = CustomGen(test_scaled[:-pre_len+1,:,:,:], test_scaled[pre_len-1:,:,:,:], length=seq_len, batch_size = 1)

x, y = train_generator[0]
print('Train: %d' % len(train_generator))
print(x.shape)
print(y.shape)

x, y = valid_generator[0]
print('Valid: %d' % len(valid_generator))
print(x.shape)
print(y.shape)

x, y = test_generator[0]
print('Test: %d' % len(test_generator))
print(x.shape)
print(y.shape)

## Prepare the model

In [ ]:
keras.backend.clear_session() # clear previous model
model = Sequential()
model.add(TimeDistributed(Conv2D(32, (3,3), activation='relu', padding = 'same'), input_shape=(None, num_nodes, num_vars, 1)))
model.add(TimeDistributed(MaxPooling2D(pool_size = 2, padding = 'same')))
model.add(TimeDistributed(Conv2D(64, (5,3), activation='relu', padding = 'same')))
model.add(TimeDistributed(Flatten())) 
model.add(TimeDistributed(Dense(128, activation='relu'))) 

model.add(LSTM(16, activation='tanh', return_sequences=False)) # change by 16, 32, 64, 128, 256
model.add(Dense(256, activation='sigmoid'))
model.add(Dense(num_nodes, activation='sigmoid'))

model.compile(optimizer='adam', loss='mae',metrics=["mse"])

#from keras.callbacks import ModelCheckpoint
#mc = ModelCheckpoint('models_MFSTC-CNN-LSTM/24_1_32_16/best_model.h5', monitor='val_loss', mode='min', verbose=0, save_best_only=True)

In [ ]:
model.summary()

### Train

In [ ]:
steps_per_epoch = int(len(train_generator) / 32)
validation_steps = int(len(valid_generator) / 32)

train_start = datetime.datetime.now()
history = model.fit_generator(
    train_generator,
    epochs=100,
    steps_per_epoch=steps_per_epoch,
    shuffle=True,
    #verbose=0,
    validation_data=valid_generator,
    validation_steps=validation_steps#, callbacks = [mc]
)
train_end = datetime.datetime.now()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.ylabel('loss')
plt.legend(['train','validation'])
plt.show()

plt.plot(history.history['mse'])
plt.plot(history.history['val_mse'])
plt.ylabel('mse')
plt.legend(['train','validation'])
plt.show()

In [ ]:
print(
    "Train loss: ",
    history.history["loss"][-1],
    "\nValid loss:",
    history.history["val_loss"][-1],
    "\nElapsed time: ",
    train_end-train_start,
)

## Evaluate

In [ ]:
##If loading best model
#from tensorflow import keras
#model=keras.models.load_model("models_MFSTC-CNN-LSTM/24_1_32_16/best_model.h5")
#model.summary()

In [ ]:
ythat = model.predict(train_generator)
yhat = model.predict(test_generator)
yht = model.predict(valid_generator)

In [ ]:
trainY = []
for i in range(len(train_generator)):
    x, y = train_generator[i]
    trainY.append(y[0])

trainY = np.squeeze(np.array(trainY))
trainY.shape

In [ ]:
testY = []
for i in range(len(test_generator)):
    x, y = test_generator[i]
    testY.append(y[0])

testY = np.squeeze(np.array(testY))
testY.shape

In [ ]:
validY = []
for i in range(len(valid_generator)):
    x, y = valid_generator[i]
    validY.append(y[0])

validY = np.squeeze(np.array(validY))
validY.shape

In [ ]:
## Rescale values
max_speed = np.reshape(np.squeeze(train_data).max(0).max(0), (6, 1))[0]
min_speed = np.reshape(np.squeeze(train_data).min(0).min(0), (6, 1))[0]

## actual values
train_rescref = np.array(trainY * max_speed)
test_rescref = np.array(testY * max_speed)
valid_rescref = np.array(validY * max_speed)

In [ ]:
train_rescpred = np.array((ythat) * max_speed)
test_rescpred = np.array((yhat) * max_speed)
valid_rescpred = np.array((yht) * max_speed)

In [ ]:
# Load the naïve values (beware to change the files depending on the pre_len value)
trainnpredc = pd.read_csv('C:/Users/lgarc/OneDrive/Documentos/AA UNI/Master/TFM/3. Review of the state of the art/trainnpredc_2.txt', sep=' ', header=None).to_numpy()
testnpredc = pd.read_csv('C:/Users/lgarc/OneDrive/Documentos/AA UNI/Master/TFM/3. Review of the state of the art/testnpredc_2.txt', sep=' ', header=None).to_numpy()
validnpredc = pd.read_csv('C:/Users/lgarc/OneDrive/Documentos/AA UNI/Master/TFM/3. Review of the state of the art/validnpredc_2.txt', sep=' ', header=None).to_numpy()

In [ ]:
## Performance measures

from sklearn.metrics import r2_score, explained_variance_score#, mean_absolute_percentage_error
from sklearn.metrics.regression import _check_reg_targets, check_consistent_length

def mean_absolute_percentage_error(y_true, y_pred,
                                   sample_weight=None,
                                   multioutput='uniform_average'):

    y_type, y_true, y_pred, multioutput = _check_reg_targets(
        y_true, y_pred, multioutput)
    check_consistent_length(y_true, y_pred, sample_weight)
    epsilon = np.finfo(np.float64).eps
    mape = np.abs(y_pred - y_true) / np.maximum(np.abs(y_true), epsilon)
    output_errors = np.average(mape,
                               weights=sample_weight, axis=0)
    if isinstance(multioutput, str):
        if multioutput == 'raw_values':
            return output_errors
        elif multioutput == 'uniform_average':
            # pass None as weights to np.average: uniform mean
            multioutput = None

    return np.average(output_errors, weights=multioutput)

On the training set

In [ ]:
# Metrics
seg_nmael = [] #naïve mae
seg_nmsel = [] #naïve mse
seg_nmape = [] #naïve mape
seg_nacc = []  #naïve accuracy
seg_nR2 = []   #naïve coeff of determination
seg_nvar = []  #naïve coeff of variance score

seg_mael = []  #mae
seg_masel = [] #mase
seg_msel = []  #mse
seg_mape = []  #mape
seg_acc = []   #accuracy
seg_R2 = []    #coeff of determination
seg_var = []   #coeff of variance score

kk=num_nodes
for j in range(kk):
    
    ## NAIVE
    # Mean Square Error 
    seg_nmsel.append(np.mean(np.square(train_rescref.T[j] - trainnpredc.T[j]))) 
        
    # Mean Absolute Error 
    seg_nmael.append(np.mean(np.abs(train_rescref.T[j] - trainnpredc.T[j])))  

    # Mean Absolute Percentage Error
    seg_nmape.append(mean_absolute_percentage_error(train_rescref.T[j],trainnpredc.T[j]))
    
    # Accuracy
    seg_nacc.append(1-np.linalg.norm(train_rescref.T[j]-trainnpredc.T[j])/np.linalg.norm(train_rescref.T[j]))
    
    # Coefficient of determination
    seg_nR2.append(r2_score(train_rescref.T[j],trainnpredc.T[j]))
    
    # Explained variance score
    seg_nvar.append(explained_variance_score(train_rescref.T[j],trainnpredc.T[j]))
    
    
    ## NN
    # Mean Absolute Error 
    seg_mael.append(np.mean(np.abs(train_rescref.T[j] - train_rescpred.T[j])))  
    
    # Mean Square Error 
    seg_msel.append(np.mean(np.square(train_rescref.T[j] - train_rescpred.T[j]))) 
       
    # Mean Absolute Percentage Error
    seg_mape.append(mean_absolute_percentage_error(train_rescref.T[j],train_rescpred.T[j]))
    
    # Accuracy
    seg_acc.append(1-np.linalg.norm(train_rescref.T[j]-train_rescpred.T[j])/np.linalg.norm(train_rescref.T[j]))
    
    # Coefficient of determination
    seg_R2.append(r2_score(train_rescref.T[j],train_rescpred.T[j]))
    
    # Explained variance score
    seg_var.append(explained_variance_score(train_rescref.T[j],train_rescpred.T[j]))
    
    if seg_nmael[-1] != 0:
        seg_masel.append(
            seg_mael[-1] / seg_nmael[-1]
        )  # Ratio of the two: Mean Absolute Scaled Error
    else:
        seg_masel.append(np.NaN)

print("Total (ave) MAE for NN: " + str(np.mean(np.array(seg_mael))))
print("Total (ave) RMSE for NN: " + str(np.mean(np.sqrt(np.array(seg_msel)))))
print("Total (ave) MAPE for NN: " + str(np.mean(np.array(seg_mape))))
print("Total (ave) Accuracy for NN: " + str(np.mean(np.array(seg_acc))))
print("Total (ave) R2 for NN: " + str(np.mean(np.array(seg_R2))))
print("Total (ave) Var for NN: " + str(np.mean(np.array(seg_var))))
print("Total (ave) MASE for per-segment NN/naive MAE: " + str(np.nanmean(np.array(seg_masel))))
print("...note that MASE<1 (for a given segment) means that the NN prediction is better than the naive prediction.")

print("\nTotal (ave) MAE for naive prediction: " + str(np.mean(np.array(seg_nmael))))
print("Total (ave) RMSE for naive prediction: " + str(np.mean(np.sqrt(np.array(seg_nmsel)))))
print("Total (ave) MAPE for naive prediction: " + str(np.mean(np.array(seg_nmape))))
print("Total (ave) Accuracy for naive prediction: " + str(np.mean(np.array(seg_nacc))))
print("Total (ave) R2 for naive prediction: " + str(np.mean(np.array(seg_nR2))))
print("Total (ave) Var for naive prediction: " + str(np.mean(np.array(seg_nvar))))

On the validation set

In [ ]:
# Metrics
seg_nmael = [] #naïve mae
seg_nmsel = [] #naïve mse
seg_nmape = [] #naïve mape
seg_nacc = []  #naïve accuracy
seg_nR2 = []   #naïve coeff of determination
seg_nvar = []  #naïve coeff of variance score

seg_mael = []  #mae
seg_masel = [] #mase
seg_msel = []  #mse
seg_mape = []  #mape
seg_acc = []   #accuracy
seg_R2 = []    #coeff of determination
seg_var = []   #coeff of variance score

kk=num_nodes
for j in range(kk):

    ## NAIVE
    # Mean Square Error
    seg_nmsel.append(np.mean(np.square(valid_rescref.T[j] - validnpredc.T[j])))

    # Mean Absolute Error
    seg_nmael.append(np.mean(np.abs(valid_rescref.T[j] - validnpredc.T[j])))

    # Mean Absolute Percentage Error
    seg_nmape.append(mean_absolute_percentage_error(valid_rescref.T[j], validnpredc.T[j]))

    # Accuracy
    seg_nacc.append(1 - np.linalg.norm(valid_rescref.T[j] - validnpredc.T[j]) / np.linalg.norm(valid_rescref.T[j]))

    # Coefficient of determination
    seg_nR2.append(r2_score(valid_rescref.T[j], validnpredc.T[j]))

    # Explained variance score
    seg_nvar.append(explained_variance_score(valid_rescref.T[j], validnpredc.T[j]))

    ## NN
    # Mean Absolute Error
    seg_mael.append(np.mean(np.abs(valid_rescref.T[j] - valid_rescpred.T[j])))

    # Mean Square Error
    seg_msel.append(np.mean(np.square(valid_rescref.T[j] - valid_rescpred.T[j])))

    # Mean Absolute Percentage Error
    seg_mape.append(mean_absolute_percentage_error(valid_rescref.T[j], valid_rescpred.T[j]))

    # Accuracy
    seg_acc.append(1 - np.linalg.norm(valid_rescref.T[j] - valid_rescpred.T[j]) / np.linalg.norm(valid_rescref.T[j]))

    # Coefficient of determination
    seg_R2.append(r2_score(valid_rescref.T[j], valid_rescpred.T[j]))

    # Explained variance score
    seg_var.append(explained_variance_score(valid_rescref.T[j], valid_rescpred.T[j]))

    if seg_nmael[-1] != 0:
        seg_masel.append(seg_mael[-1] / seg_nmael[-1])  # Ratio of the two: Mean Absolute Scaled Error
    else:
        seg_masel.append(np.NaN)

print("Total (ave) MAE for NN: " + str(np.mean(np.array(seg_mael))))
print("Total (ave) RMSE for NN: " + str(np.mean(np.sqrt(np.array(seg_msel)))))
print("Total (ave) MAPE for NN: " + str(np.mean(np.array(seg_mape))))
print("Total (ave) Accuracy for NN: " + str(np.mean(np.array(seg_acc))))
print("Total (ave) R2 for NN: " + str(np.mean(np.array(seg_R2))))
print("Total (ave) Var for NN: " + str(np.mean(np.array(seg_var))))
print("Total (ave) MASE for per-segment NN/naive MAE: " + str(np.nanmean(np.array(seg_masel))))
print("...note that MASE<1 (for a given segment) means that the NN prediction is better than the naive prediction.")

print("\nTotal (ave) MAE for naive prediction: " + str(np.mean(np.array(seg_nmael))))
print("Total (ave) RMSE for naive prediction: " + str(np.mean(np.sqrt(np.array(seg_nmsel)))))
print("Total (ave) MAPE for naive prediction: " + str(np.mean(np.array(seg_nmape))))
print("Total (ave) Accuracy for naive prediction: " + str(np.mean(np.array(seg_nacc))))
print("Total (ave) R2 for naive prediction: " + str(np.mean(np.array(seg_nR2))))
print("Total (ave) Var for naive prediction: " + str(np.mean(np.array(seg_nvar))))

On the test set

In [ ]:
# Metrics
seg_nmael = [] #naïve mae
seg_nmsel = [] #naïve mse
seg_nmape = [] #naïve mape
seg_nacc = []  #naïve accuracy
seg_nR2 = []   #naïve coeff of determination
seg_nvar = []  #naïve coeff of variance score

seg_mael = []  #mae
seg_masel = [] #mase
seg_msel = []  #mse
seg_mape = []  #mape
seg_acc = []   #accuracy
seg_R2 = []    #coeff of determination
seg_var = []   #coeff of variance score

kk=num_nodes
for j in range(kk):

    ## NAIVE
    # Mean Square Error
    seg_nmsel.append(np.mean(np.square(test_rescref.T[j] - testnpredc.T[j])))

    # Mean Absolute Error
    seg_nmael.append(np.mean(np.abs(test_rescref.T[j] - testnpredc.T[j])))

    # Mean Absolute Percentage Error
    seg_nmape.append(mean_absolute_percentage_error(test_rescref.T[j], testnpredc.T[j]))

    # Accuracy
    seg_nacc.append(1 - np.linalg.norm(test_rescref.T[j] - testnpredc.T[j]) / np.linalg.norm(test_rescref.T[j]))

    # Coefficient of determination
    seg_nR2.append(r2_score(test_rescref.T[j], testnpredc.T[j]))

    # Explained variance score
    seg_nvar.append(explained_variance_score(test_rescref.T[j], testnpredc.T[j]))

    ## NN
    # Mean Absolute Error
    seg_mael.append(np.mean(np.abs(test_rescref.T[j] - test_rescpred.T[j])))

    # Mean Square Error
    seg_msel.append(np.mean(np.square(test_rescref.T[j] - test_rescpred.T[j])))

    # Mean Absolute Percentage Error
    seg_mape.append(mean_absolute_percentage_error(test_rescref.T[j], test_rescpred.T[j]))

    # Accuracy
    seg_acc.append(1 - np.linalg.norm(test_rescref.T[j] - test_rescpred.T[j]) / np.linalg.norm(test_rescref.T[j]))

    # Coefficient of determination
    seg_R2.append(r2_score(test_rescref.T[j], test_rescpred.T[j]))

    # Explained variance score
    seg_var.append(explained_variance_score(test_rescref.T[j], test_rescpred.T[j]))

    if seg_nmael[-1] != 0:
        seg_masel.append(seg_mael[-1] / seg_nmael[-1])  # Ratio of the two: Mean Absolute Scaled Error
    else:
        seg_masel.append(np.NaN)

print("Total (ave) MAE for NN: " + str(np.mean(np.array(seg_mael))))
print("Total (ave) RMSE for NN: " + str(np.mean(np.sqrt(np.array(seg_msel)))))
print("Total (ave) MAPE for NN: " + str(np.mean(np.array(seg_mape))))
print("Total (ave) Accuracy for NN: " + str(np.mean(np.array(seg_acc))))
print("Total (ave) R2 for NN: " + str(np.mean(np.array(seg_R2))))
print("Total (ave) Var for NN: " + str(np.mean(np.array(seg_var))))
print("Total (ave) MASE for per-segment NN/naive MAE: " + str(np.nanmean(np.array(seg_masel))))
print("...note that MASE<1 (for a given segment) means that the NN prediction is better than the naive prediction.")

print("\nTotal (ave) MAE for naive prediction: " + str(np.mean(np.array(seg_nmael))))
print("Total (ave) RMSE for naive prediction: " + str(np.mean(np.sqrt(np.array(seg_nmsel)))))
print("Total (ave) MAPE for naive prediction: " + str(np.mean(np.array(seg_nmape))))
print("Total (ave) Accuracy for naive prediction: " + str(np.mean(np.array(seg_nacc))))
print("Total (ave) R2 for naive prediction: " + str(np.mean(np.array(seg_nR2))))
print("Total (ave) Var for naive prediction: " + str(np.mean(np.array(seg_nvar))))

In [ ]:
mypath = 'metrics/MFSTC-CNN-LSTM_24_1_32_16/'    #seq_len _ pre_len _ batch_size _ hidden_units

a_file = open(mypath+"MAE.txt", "w")
for row in np.matrix(seg_mael):
    np.savetxt(a_file, row)
a_file.close()

a_file = open(mypath+"RMSE.txt", "w")
for row in np.matrix(seg_msel):
    np.savetxt(a_file, row)
a_file.close()

a_file = open(mypath+"MASE.txt", "w")
for row in np.matrix(seg_masel):
    np.savetxt(a_file, row)
a_file.close()

a_file = open(mypath+"ACC.txt", "w")
for row in np.matrix(seg_acc):
    np.savetxt(a_file, row)
a_file.close()

a_file = open(mypath+"R2.txt", "w")
for row in np.matrix(seg_R2):
    np.savetxt(a_file, row)
a_file.close()

a_file = open(mypath+"EVS.txt", "w")
for row in np.matrix(seg_var):
    np.savetxt(a_file, row)
a_file.close()

In [ ]:
##Result visualization
fig1 = plt.figure(figsize=(15, 8))
#    ax1 = fig1.add_subplot(1,1,1)
a_pred = test_rescpred[0:, 20]
a_true = test_rescref[0:, 20]
plt.plot(a_true, "tab:green", label="true",linewidth=0.5)
plt.plot(a_pred, "tab:blue", label="prediction",linewidth=0.5)
plt.xlabel("time")
plt.ylabel("speed")
plt.legend(loc="best", fontsize=10)
plt.show()

## Save results

In [ ]:
mypath = 'models_MFSTC-CNN-LSTM/24_1_32_16/'    #seq_len _ pre_len _ batch_size _ hidden_units
model.save(mypath + 'model.h5')

a_file = open(mypath+"train_rescref.txt", "w")
for row in np.matrix(train_rescref):
    np.savetxt(a_file, row)
a_file.close()

a_file = open(mypath+"train_rescpred.txt", "w")
for row in np.matrix(train_rescpred):
    np.savetxt(a_file, row)
a_file.close()

a_file = open(mypath+"valid_rescref.txt", "w")
for row in np.matrix(valid_rescref):
    np.savetxt(a_file, row)
a_file.close()

a_file = open(mypath+"valid_rescpred.txt", "w")
for row in np.matrix(valid_rescpred):
    np.savetxt(a_file, row)
a_file.close()

a_file = open(mypath+"test_rescref.txt", "w")
for row in np.matrix(test_rescref):
    np.savetxt(a_file, row)
a_file.close()

a_file = open(mypath+"test_rescpred.txt", "w")
for row in np.matrix(test_rescpred):
    np.savetxt(a_file, row)
a_file.close()